In [13]:
import h2o
from h2o.frame import H2OFrame
import pandas as pd
from h2o.automl import H2OAutoML
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,16 mins 41 secs
H2O_cluster_timezone:,Europe/Istanbul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.9
H2O_cluster_version_age:,2 months and 11 days
H2O_cluster_name:,H2O_from_python_merve_rru9pi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.740 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [14]:

df = pd.read_csv("C:\\Users\\merve\\Desktop\\staj_lotus_ai\\EtSut_ML_Projects\\01_classification_Food_Adulteration\\data\\food_adulteration_data.csv")

print(df.head())
print(df.info())

   adulteration_id product_name   brand   category             adulterant  \
0                1       Butter  BrandB       Meat  Artificial sweeteners   
1                2      Chicken  BrandC      Dairy        Coloring agents   
2                3       Yogurt  BrandC       Meat  Artificial sweeteners   
3                4         Wine  BrandB  Beverages        Coloring agents   
4                5        Bread  BrandD      Dairy                  Water   

  detection_date          detection_method  severity health_risk  \
0      5/11/2024  Microbiological Analysis  Moderate         Low   
1      5/23/2024        Sensory Evaluation    Severe      Medium   
2      2/17/2024        Sensory Evaluation    Severe        High   
3      5/16/2024              Spectroscopy     Minor      Medium   
4       6/6/2024         Chemical Analysis    Severe      Medium   

             action_taken  
0          Product Recall  
1          Warning Issued  
2  Investigation Launched  
3          Produ

In [15]:


# Pandas DataFrame'i H2OFrame'e çevir
hf = h2o.H2OFrame(df)

# Target sütununu tanımla
target = "health_risk"

# Eğer sınıflandırma yapıyorsan (kategorik target):
hf[target] = hf[target].asfactor()

# Feature sütunları (target hariç tüm sütunlar)
features = hf.columns
features.remove(target)

print("Target:", target)
print("Features:", features)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Target: health_risk
Features: ['adulteration_id', 'product_name', 'brand', 'category', 'adulterant', 'detection_date', 'detection_method', 'severity', 'action_taken']


In [16]:


# Train / Test split (75% train, 25% test)
train, test = hf.split_frame(ratios=[0.75], seed=42)

# AutoML nesnesi
aml = H2OAutoML(
    max_models=10,        # denenecek model sayısı
    max_runtime_secs=300, # 5 dakika
    seed=42
)

# AutoML'i eğit
aml.train(x=features, y=target, training_frame=train)

# En iyi modeli al
best_model = aml.leader

print(best_model)


AutoML progress: |█
22:45:03.422: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_BestOfFamily_1_AutoML_3_20260204_224503


Model Summary for Stacked Ensemble: 
key                                        value
-----------------------------------------  ----------------
Stacking strategy                          cross_validation
Number of base models (used / total)       0/5
# GBM base models (used / total)           0/1
# DeepLearning base models (used / total)  0/1
# DRF base models (used / total)           0/2
# GLM base models (used / total)           0/1
Metalearner algorithm                      GLM
Metalearner fold assignment scheme         Random
Metalearner nfolds                         5
Metalearner fold_column
Custom metalearner hyperparameters         None

ModelMetricsMultinomialGLM: stackedensemble
** Reported 

In [17]:
# Test seti üzerinde tahmin yap
performance = best_model.model_performance(test)

# Performans sonuçlarını yazdır
print(performance)
print("\n" + "="*50)

# Confusion Matrix
cm = performance.confusion_matrix()
print("\nConfusion Matrix:")
print(cm)

# Accuracy değerini confusion matrix'ten al
if hasattr(cm, 'to_list'):
    cm_table = cm.to_list()
    if cm_table:
        # Son satırda genellikle toplam accuracy bulunur
        print("\n" + "="*50)
        print("Accuracy:", cm_table[-1][-1])

# Alternatif olarak, doğrudan metrik değerleri
print("\n" + "="*50)
print("Additional Metrics:")
print(f"Logloss: {performance.logloss()}")
print(f"Mean Per-Class Error: {performance.mean_per_class_error()}")

# AUC (eğer binary classification ise)
try:
    print(f"AUC: {performance.auc()}")
except:
    print("AUC not available (likely multiclass classification)")


ModelMetricsMultinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.44299321346082265
RMSE: 0.6655773534765307
LogLoss: 1.0955266954590837
Null degrees of freedom: 255
Residual degrees of freedom: 253
Null deviance: 560.9096680750506
Residual deviance: 560.9096680750506
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
High    Low    Medium    Error     Rate
------  -----  --------  --------  ---------
0       80     0         1         80 / 80
0       101    0         0         0 / 101
0       75     0         1         75 / 75
0       256    0         0.605469  155 / 256

Top-3 Hit Ratios: 
k    hit_ratio
---  ---------

In [18]:
# En iyi model ve accuracy bilgisi
print("="*60)
print("EN İYİ MODEL BİLGİSİ")
print("="*60)
print(f"Model Adı: {best_model.model_id}")
print(f"Model Tipi: {best_model.algo}")
print("\n")

# Leaderboard - Tüm modelleri göster
print("="*60)
print("TÜM MODELLERİN PERFORMANSI (LEADERBOARD)")
print("="*60)
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))
print("\n")

# Test seti performansı
print("="*60)
print("TEST SETİ PERFORMANSI")
print("="*60)
performance = best_model.model_performance(test)

# Confusion Matrix'ten accuracy al
cm = performance.confusion_matrix()
print(cm)
print("\n")

# Accuracy değerini confusion matrix'ten hesapla
if hasattr(cm, 'to_list'):
    cm_table = cm.to_list()
    if cm_table and len(cm_table) > 1:
        # Error Rate'ten Accuracy hesapla
        error_rate = cm_table[-1][-1]
        accuracy = 1 - error_rate
        print(f"Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
        print(f"Test Error Rate: {error_rate:.4f}")

print("\n" + "="*60)
print("DİĞER METRİKLER")
print("="*60)
print(f"Logloss: {performance.logloss():.4f}")
print(f"Mean Per-Class Error: {performance.mean_per_class_error():.4f}")

try:
    print(f"AUC: {performance.auc():.4f}")
except:
    print("AUC: Multiclass classification için mevcut değil")

EN İYİ MODEL BİLGİSİ
Model Adı: StackedEnsemble_BestOfFamily_1_AutoML_3_20260204_224503
Model Tipi: stackedensemble


TÜM MODELLERİN PERFORMANSI (LEADERBOARD)
model_id                                                   mean_per_class_error    logloss      rmse       mse
StackedEnsemble_BestOfFamily_1_AutoML_3_20260204_224503                0.666667    1.09827  0.666497  0.444218
StackedEnsemble_AllModels_1_AutoML_3_20260204_224503                   0.666667    1.09827  0.666497  0.444218
GBM_4_AutoML_3_20260204_224503                                         0.67562     1.19748  0.686295  0.471
DeepLearning_1_AutoML_3_20260204_224503                                0.680241    1.18019  0.683738  0.467498
GBM_5_AutoML_3_20260204_224503                                         0.682084    1.19037  0.684536  0.46859
GBM_2_AutoML_3_20260204_224503                                         0.690251    1.19745  0.686177  0.470839
GBM_3_AutoML_3_20260204_224503                                      

In [19]:
# Target dağılımı
df[target].value_counts(normalize=True) * 100

health_risk
Low       35.8
High      32.5
Medium    31.7
Name: proportion, dtype: float64